# 【先行研究：高値と安値を予測する問題】
株式市場では、高値と安値のレンジを予測する先行研究があります。これはマーケットメイカー（値付け業者）が取引板の上下に指値を出し、高値と安値のレンジを予測して収益を最大化するための研究となります。

「執行戦略と取引コストに関する研究の進展」杉原

https://www.imes.boj.or.jp/research/papers/japanese/kk31-1-8.pdf

この論文中の式(21)に表されるように、基本的に高値、安値のレンジはボラティリティの関数となります。実際に、チュートリアルのtrainデータ、valデータ、testデータでも、特徴量"volatility_1month"だけでかなりの精度を得ることができます。

In [2]:
import pickle
import datetime
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, DayLocator, DateFormatter
from mpl_finance import candlestick2_ohlc, volume_overlay

from IPython.display import Image

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from tqdm.auto import tqdm

%matplotlib inline
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.width = 120
warnings.simplefilter('ignore')

In [3]:
def pickle_dump(obj, path):
    with open(path, mode='wb') as f:
        pickle.dump(obj,f)

def pickle_load(path):
    with open(path, mode='rb') as f:
        data = pickle.load(f)
        return data

In [4]:
# チュートリアルのtrain_X, val_X, test_X, train_y, val_y, test_y　私はpickleで保存しています
# ""の中身はご自身の環境に合わせて定義してください
train_X = pickle_load("...")
val_X = pickle_load("...")
test_X = pickle_load("...")
train_y = pickle_load("...")
val_y = pickle_load("...")
test_y = pickle_load("...")

In [5]:
# データセットの読み込み
# ""の中身はご自身の環境に合わせて定義してください
dataset_dir="..."
inputs = {
    "stock_list": f"{dataset_dir}/stock_list.csv",
    "stock_price": f"{dataset_dir}/stock_price.csv",
    "stock_fin": f"{dataset_dir}/stock_fin.csv",
    "stock_labels": f"{dataset_dir}/stock_labels.csv",
}

dfs = {}
for k, v in inputs.items():
    dfs[k] = pd.read_csv(v)

In [14]:
def get_score_of_volatility(df_X, df_y):
    tmp_df = df_X["label_high_20"].copy()
    tmp_df["label_high"] = df_y["label_high_20"]
    tmp_df["label_low"]  = df_y["label_low_20"]
    rho_high = tmp_df["volatility_1month"].rank().corr(tmp_df["label_high"].rank())
    rho_low = -tmp_df["volatility_1month"].rank().corr(tmp_df["label_low"].rank())
    score = (rho_high - 1)**2 + (rho_low - 1)**2
    return score

## volatility_1monthのみで予測したtrainデータのスコア

In [15]:
get_score_of_volatility(train_X, train_y)

1.290669731596573

## volatility_1monthのみで予測したvalデータのスコア

In [16]:
get_score_of_volatility(val_X, val_y)

1.1433213619473521

## volatility_1monthのみで予測したtestデータのスコア

In [17]:
get_score_of_volatility(test_X, test_y)

1.2107673177457172